### 総合演習データパイプライン

* 入力：CSVファイル ("C:\Users\Admin\ILP\Python-AI\raw_data.csv")
* 出力：

In [1]:
# install
%pip install pandas

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------- ----------------------------- 2.9/11.0 MB 15.8 MB/s eta 0:00:01
   ------------------------------------ --- 10.0/11.0 MB 25.2 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 19.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   --------------------- ------------------ 6.8/12.7 MB 34.9 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.7 MB 29.7 MB/s eta 0:00:01
   ---------------------------------------- 12.7/12.7 MB 19.6 MB/s eta 0:00:00

   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
  

In [4]:
#必要なライブラリ
import pandas as pd
import numpy as np

In [56]:
df = pd.read_csv('raw_data.csv',encoding='utf-8')
df.head()

,日付,予約件数,予約人数,来客数,総杯数,売上合計(円),ペールエール(本),ペールエール(円),ラガー(本),ラガー(円),IPA(本),IPA(円),ホワイトビール(本),ホワイトビール(円),黒ビール(本),黒ビール(円),フルーツビール(本),フルーツビール(円)
0,2024/04/01,NaN,NaN,21,33,"31,800",10,"10,000",10,"8,000",5,"4,500",5,"4,500",4,"4,800",NaN,NaN
1,2024/04/02,NaN,NaN,13,25,"23,300",8,"8,000",7,"5,600",3,"2,700",4,"3,600",2,"2,400",1,"1,000"
2,2024/04/03,NaN,NaN,20,36,"33,600",7,"7,000",8,"6,400",6,"5,400",6,"5,400",2,"2,400",7,"7,000"
3,2024/04/04,NaN,NaN,17,27,"24,600",6,"6,000",9,"7,200",4,"3,600",4,"3,600",1,"1,200",3,"3,000"
4,2024/04/05,2,5,29,61,"56,500",15,"15,000",18,"14,400",9,"8,100",8,"7,200",4,"4,800",7,"7,000"


In [57]:
# 正しい列名を取得（1行目のヘッダーとして読み込まれたもの）
header_row = df.columns.tolist()
# ヘッダーとまったく同じ値を持つ行を探して削除
df = df[~df.apply(lambda row: row.tolist() == header_row, axis=1)]
# 日付をdatetime型に変換
df['日付'] = pd.to_datetime(df['日付'], format='%Y/%m/%d', errors='coerce')
# 数値列の返還
numeric_cols = df.columns.drop('日付')
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')



In [58]:
df.dtypes

日付            datetime64[ns]
予約件数                 float64
予約人数                 float64
来客数                  float64
総杯数                  float64
売上合計(円)              float64
ペールエール(本)            float64
ペールエール(円)            float64
ラガー(本)               float64
ラガー(円)               float64
IPA(本)               float64
IPA(円)               float64
ホワイトビール(本)           float64
ホワイトビール(円)           float64
黒ビール(本)              float64
黒ビール(円)              float64
フルーツビール(本)           float64
フルーツビール(円)           float64
dtype: object

In [59]:
df.isnull().sum()

日付              0
予約件数          296
予約人数          296
来客数             1
総杯数             1
売上合計(円)       314
ペールエール(本)       5
ペールエール(円)     315
ラガー(本)          8
ラガー(円)        256
IPA(本)         29
IPA(円)        249
ホワイトビール(本)     58
ホワイトビール(円)    263
黒ビール(本)        82
黒ビール(円)       315
フルーツビール(本)    121
フルーツビール(円)    314
dtype: int64

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 315 entries, 0 to 325
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   日付          315 non-null    datetime64[ns]
 1   予約件数        19 non-null     float64       
 2   予約人数        19 non-null     float64       
 3   来客数         314 non-null    float64       
 4   総杯数         314 non-null    float64       
 5   売上合計(円)     1 non-null      float64       
 6   ペールエール(本)   310 non-null    float64       
 7   ペールエール(円)   0 non-null      float64       
 8   ラガー(本)      307 non-null    float64       
 9   ラガー(円)      59 non-null     float64       
 10  IPA(本)      286 non-null    float64       
 11  IPA(円)      66 non-null     float64       
 12  ホワイトビール(本)  257 non-null    float64       
 13  ホワイトビール(円)  52 non-null     float64       
 14  黒ビール(本)     233 non-null    float64       
 15  黒ビール(円)     0 non-null      float64       
 16  フルーツビール(本)  194 non-null    flo

In [61]:
# 欠損が「少ない行」を削除（5〜10行程度の欠損）
df = df.dropna(subset=[
    '来客数', '総杯数', 'ペールエール(本)', 'ラガー(本)'
])
# 欠損が「多すぎる列」を削除（90%以上欠損）
df = df.drop(columns=[
    '予約件数', '予約人数', '売上合計(円)', 'ペールエール(円)', '黒ビール(円)', 'フルーツビール(円)'
])
# 中程度の欠損（20%〜50%程度） → 平均や 0 で埋める
df['フルーツビール(本)'] = df['フルーツビール(本)'].fillna(df['フルーツビール(本)'].mean())
df['IPA(本)'] = df['IPA(本)'].fillna(df['IPA(本)'].mean())
df['ラガー(円)'] = df['ラガー(円)'].fillna(df['ラガー(円)'].mean())
df['IPA(円)'] = df['IPA(円)'].fillna(df['IPA(本)'].mean())
df['ホワイトビール(円)'] = df['ホワイトビール(円)'].fillna(df['ホワイトビール(円)'].mean())
df['ホワイトビール(本)'] = df['ホワイトビール(本)'].fillna(df['ホワイトビール(本)'].mean())
df['黒ビール(本)'] = df['黒ビール(本)'].fillna(df['黒ビール(本)'].mean())


In [62]:
df.isnull().sum()

日付            0
来客数           0
総杯数           0
ペールエール(本)     0
ラガー(本)        0
ラガー(円)        0
IPA(本)        0
IPA(円)        0
ホワイトビール(本)    0
ホワイトビール(円)    0
黒ビール(本)       0
フルーツビール(本)    0
dtype: int64

In [64]:
df.to_json("sales_data.json")